Progetto: **Classificazione di testi descrittivi per destinazione d'uso, inerenti ai beni sottoposti ad aste giudiziarie italiane.**

Studente: **Alessandro Monolo** | *10439147*

Relatore: Marco Brambilla

Referente aziendale: Simone Redaelli

Master: Data Science & Artificial Intelligence

Università: Politecnico di Milano

<hr style="border:1px solid black">

## Seconda parte di Feature Engineering e Pre - Processing sulla variabile testuale

- **1.** **<u>Feature enginering</u>**:
    - **1.1** **Creazione di una nuova colonna** che conta il numero di **StopWords** per descrizione;
    - **1.2** **Creazione di una nuova colonna** che conta il numero di **Cifre** presenti per descrizione;
    - **1.3** **Creazione di una nuova colonna** che conta il numero di **Upper case words** per descrizione;


- **2.** **<u>Pre-Processing sulla variabile testuale:</u>**
    - Per la fase di pre-processing sulla variabile testuale invece, ho eseguito finora la pulizia del testo, togliendo:
        - **2.1** **e-mails**;
        - **2.2** **URLs**;
        - **2.3** **Doppi spazi o spazi multipli**;
        - **2.4** **Lower casa only**;
        - **2.5** **Punctuation**;
        - **2.6** **Stopwords**;
        - **2.7** **Caratteri accentati**;
        - **2.8** **Di nuovo - Doppi spazi o spazi multipli**;
        - **2.9** **Rimozione delle parole troppo corte, sotto i 2 caratteri**;
        - **2.10** **Rimozione parole non comuni**;
        


- **3. <u>Pre-processo la variabile testuale attraverso la libreria denominata Spacy per la fase di Lemmatization</u>**


- **4.** **<u>Fase di Pre-Processign per la Target Variable "Destinazione d'uso", applicando LabelEncoder</u>**;


- **5.** **<u>Filtro il data frame</u>**;


- **6.** **<u>Conclusioni</u>**


- **7.** **<u>Export final CSV</u>**

<hr style="border:1px solid black">

**Importo le librerie che mi servono:**

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew
import math
import plotly.express as px
import warnings
from matplotlib import cm
from PIL import Image
import string
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
import spacy
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

#### Set pandas options:

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 1000
pd.options.display.float_format = '{:.4f}'.format
pd.options.mode.chained_assignment = None

**Importo file CSV**

In [3]:
df = pd.read_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\10_Dataset_Pre_Processing_Var_Text\\Dataset_Pre_Processing_Var_Text.csv")

### 1 - Feature Enginering

- In questo caso, come per le altre colonne create dalla variabile testuale, utilizzo il campo "Descrizione_Bene_Clean", ovvero il campo descrittivo senza i simboli strani, ma con ancora sia la punctuation sia le capital letters.

#### 1.1 Conto il numero di stop words per descrizione

In [4]:
# Instanzio il vocabolario delle stopwords - Italian:
stop_words = set(stopwords.words('italian'))
# Creo la nuova colonna:
df['N_Stop_Words'] = df["Descrizione_Bene_Clean"].apply(lambda x : len([t for t in x.split() if t in stop_words]))

#### 1.2 Conto il numero di cifre presenti per descrizione

In [5]:
# Creo la nuova colonna:
df["N_Numeri"] = df["Descrizione_Bene_Clean"].apply(lambda x: len([t for t in x.split() if t.isdigit()]))

#### 1.3 Conto il numero di stringhe in upper case per descrizione

- Per questa nuova colonna sono abbastanza dubbioso in quanto non esprimerebbe nulla. La presenza in alcuni testi delle upper case words è totalmente casuale in quanto i testi sono tutti in upper case alle volte.

In [6]:
# Creo la nuova colonna:
df["N_Upper_Word"] = df["Descrizione_Bene_Clean"].apply(lambda x: len([t for t in x.split() if t.isupper()]))

### 2 - Text Pre-Processing

- **2.1 Trovo tutte le emails nel campo descrizione, le conto ed infine le elimino dal testo:**

In [7]:
# Creo una nuova colonna che estragga tutte le email dal testo:
df["Emails"] = df["Descrizione_Bene_Clean"].apply(lambda x: re.findall(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+._-]+)', x))

# Creo un anuova colonna che conti il numero di mail dal testo:
df["N_Emails"] = df["Emails"].apply(lambda x: len(x))

# Elimino dal testo tutte le mail e salvo in una nuova colonna il testo pulito:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Clean"].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+._-]+)', '', x))

# Quantifico il numero di mails presenti nel campo descrizione del bene:
df["N_Emails"].value_counts()

0    165878
1        91
3        20
2        11
4         3
Name: N_Emails, dtype: int64

- Solo in **125 casi su 166.003**, il testo presenta almeno **una mail**.
- Solo in **23 casi** il testo della descrizione del bene presenta più di **2 mails**.

In [8]:
df_with_mails = df[df["N_Emails"] > 0]
print(df_with_mails["Destinazione_Uso"].value_counts(1), "\n")
print(df_with_mails["Destinazione_Uso"].value_counts())

RESIDENTIAL   0.7200
INDUSTRIAL    0.0960
STORAGE       0.0800
RETAIL        0.0560
LAND          0.0400
HOTEL         0.0080
Name: Destinazione_Uso, dtype: float64 

RESIDENTIAL    90
INDUSTRIAL     12
STORAGE        10
RETAIL          7
LAND            5
HOTEL           1
Name: Destinazione_Uso, dtype: int64


- Nel **72% dei casi le mails sono presenti nei beni con destinazione d'uso Residential**, mentre nel **10% dei casi le mails sono presenti nei beni Industrial**.
- Il restante **18% dei casi, le mails sono presenti nei restanti tipi** di beni.

- **2.2 Trovo tutti i links/URL/Siti Web nel campo descrizione, conto ed infine le elimino dal testo:**

In [9]:
# Creo una nuova colonna che estragga tutti i links/URL/Siti Web dal testo:
df["URL"] = df["Descrizione_Bene_Clean"].apply(lambda x: re.findall(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', x))

# Creo un anuova colonna che conti il numero di links/URL/Siti Web dal testo:
df["N_URL"] = df["URL"].apply(lambda x: len(x))

# Elimino dal testo tutti i links/URL/Siti Web e salvo in una nuova colonna il testo pulito:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w•,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))

# Quantifico il numero di links estratti dai testi:
df['N_URL'].value_counts()

0    166003
Name: N_URL, dtype: int64

- Si evince quindi che **non sono presenti link a URL nel campo descrizione** del bene.

- **2.3 Elimino tutti gli spazi doppi o spazi multipli nel campo Descrizione del bene:**

In [10]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: ' '.join(x.split()))

- **2.4 Converto tutte le parole presenti nel campo descrizione del bene in lower case only:**

In [11]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].str.lower()

- **2.5 Elimino la punctuation nel campo Descrizione del bene:**

In [12]:
# Elimino dal testo tutti gli special characters:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: re.sub(r'[^\w ]+', ' ', x))

- **2.6 Elimino tutte le stopwords presenti nel campo descrizione del bene:**

In [13]:
# Setto il vocabolario in italiano per le stopwords in italiano:
stop_words = set(stopwords.words('italian'))

# Applico la funzione stopwords alla colonna descrizione bene pulita e pre processata:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: ' '.join([t for t in x.split() if t not in stop_words]))

- **2.7 Elimino tutti i caratteri che presentano accento:**


- Importante da considerare come questa fase di pulizia, sia necessaria per eliminare tutte quelle lettere che potrebbero deformare una parola o renderla malinterpretabile durante la fase di predizione. Inoltre, la libreria NLTK.StopWords non tiene conto durante la sua applicazione di testi accentati o no.

In [14]:
# Creo una funzione che mi permetta di eliminare tutte le accento presenti nei testi
# di descrizione e le sostituisca con medesime lettere non accentuate:
def remove_accented_chars(x):
    x = unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore")
    return x
# Applico la funzione creata al campo descrizione pre processato.
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: remove_accented_chars(x))

- **2.8 Elimino di nuovo tutti gli spazi doppi o spazi multipli nel campo Descrizione del bene:**

In [15]:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: ' '.join(x.split()))

- **2.9** **Elimino le parole troppo corte, sotto i due caratteri:**

In [16]:
# Definisco una user defined function che sia in grado di eliminare tutte le parole composte da un singolo carattere:
def filter_single_char_words(text):
    words = text.split()
    filtered_words = [word for word in words if len(word) > 1]
    return ' '.join(filtered_words)

# Applico la user defined function alla colonna testuale pre-processata:
df['Descrizione_Bene_Pre_Process'] = df['Descrizione_Bene_Pre_Process'].apply(filter_single_char_words)

- **2.10** **Elimino le parole meno frequenti che rappresentano errori di battitura o errori di merge tra parole:**

In [17]:
# Metto in una variabile tutte le stringhe presenti nella colonna:
text = " ".join(df["Descrizione_Bene_Pre_Process"])
text = text.split()
# Conto le stringhe contenute e le metto in una Pandas Series:
freq_common = pd.Series(text).value_counts()
# Prendo le 200 parole meno frequenti cge rappresentano tutte parole o numeri di nessun significato:
rare_words = freq_common.tail(300)

# Applico il drop alla colonna testuale pre-processata:
df["Descrizione_Bene_Pre_Process"] = df["Descrizione_Bene_Pre_Process"].apply(lambda x: " ".join([t for t in x.split() if t not in rare_words]))

### 3 - Lemmatization text variable:

In [18]:
# Importo il vocabolario italiano:
nlp = spacy.load('it_core_news_sm')

# Creo una UDF per applicare la libreria spacy alla variabile testuale preprocessata, utilizzando Stanza:
def lemmatize_text_spacy(text_1):
    doc_1 = nlp(text_1)
    lemmatized_tokens_1 = [token.lemma_ for token in doc_1]
    return " ".join(lemmatized_tokens_1)

tqdm.pandas()
# Applico la UDF alla variabile testuale:
df['Descrizione_Bene_Lemm_Spacy'] = df['Descrizione_Bene_Pre_Process'].progress_apply(lemmatize_text_spacy)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166003/166003 [33:11<00:00, 83.35it/s]


### 4 - Fase di Pre-Processign per la Target Variable "Destinazione d'uso", applicando LabelEncoder

In [19]:
# Instanzio in una nuova variabile il modello per l'encoding della target variable:
label_encoder = LabelEncoder()

# Applico il modello di encoding alla mia target variable e salvo il risultato in una nuova colonna:
df['Destinazione_Uso_Encoded'] = label_encoder.fit_transform(df['Destinazione_Uso'])

### 5 - Filtro il data frame con solo le colonne che mi servono per la text classification

In [24]:
# Seleziono le colonne da eliminare
columns_to_drop = ['Destinazione_Uso', 
                   'Categoria_Catastale', 
                   'Tribunale', 
                   'Provincia', 
                   'Comune', 
                   'Numero_Lotto', 
                   'Numero_Vani', 
                   'Superficie', 
                   'Rapporto_Vani_Superficie', 
                   'N_Caratteri',
                   'N_Parole', 
                   'avg_word_len', 'N_Stop_Words', 
                   'N_Numeri', 'N_Upper_Word',
                   'N_Emails', 
                   'N_URL',
                   'Catasto_Fabbricati', 
                   'Descrizione_Bene',
                   'Descrizione_Bene_Clean', 
                   'Descrizione_Bene_EDA', 
                   'Emails',
                   'N_URL',
                   'URL']
# Mantengo solo le colonne che mi servono per la text classification:
df = df.drop(columns=columns_to_drop)

### 6 - Conlusioni

- Dalla variabile testuale "Descrizione_Beni_Clean", ovvero la variabile testuale inerente alle descrizioni dei lotti puliti da caratteri speciali quali "\n" e "\\", sono state create ulteriori variabili estraendo da ogni descrizione di beni:
    - Il numero di stop words;
    - Il numero di cifre menzionate;
    - Il numero di upper case words;


- Dalla medesima variabile testuale è stato poi possibile utilizzando delle regex, tutte le mail utilizzate e tutti i Link/Web Site/URLs presenti:
    - Nello specifico, le mail presenti hanno le seguenti caratteristiche:
        - Solo in 125 casi su 166.003, il testo presenta almeno una mail.
        - Solo in 23 casi il testo della descrizione del bene presenta più di 2 mails.
        - Nel 72% dei casi le mail sono presenti nei beni con destinazione d'uso Residential, mentre nel 10% dei casi le mails sono per i beni Industrial.
        - Il restante 18% dei casi, le mails sono presenti nei restanti tipi di beni.
    - Mentre per quanto riguarda i siti web, non esistono siti web all'interno di nessuna descrizione di bene presente nel data frame.


- Una volta eliminate le estrazioni menzionate sopra, dalla variabile "Descrizione_Bene_PRe_Process", sono state eseguite le seguenti manovre di pre-processing del dato testuale:
    - Eliminazione di tutti gli spazi doppi o spazi multipli;
    - Convesrione tutte le parole presenti nel campo descrizione del bene in lower case;
    - Eliminazione della punctuation;
    - Eliminazione delle stop words, avendo settato il dizionario in modalità Italiana;
    - Eliminazione di tutte le accento sui caratteri delle parole, in quanto i modelli di text classification non subiscono variazione se le stringhe presentano o meno caratteri con accenti;
    - Eliminazione di tutte le parole che presentano meno di due caratteri;
    - Infine, eliminazione delle 300 parole meno frequenti in quelle menzionate all'interno delle descrizioni dei beni;
    

- Successivamente, è stata creata una variabile testuale lemmatizzata, attraverso le libreria Spacy.
  Entrambe le variabili, quella con lemmatization e quella con solo la fase di pre-process, sono esportate nel df finale per la classificazion dei testi.
  Su tutte e due sarà fatta la classificazione e dopodichè si vedrà quale avrà ottenuto le metriche più alte.
    
    
- Come passaggio finale è stata encoddata la variabile target Destinazione d'uso attraverso la libreria LAbelEncoder per rendere numerica la variabile ed infine filtrato il data frame salvando solo le variabili che mi servono per la text classification;

### 7 - Export final CSV

In [26]:
df_csv = df.to_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\11_Dataset_TFIDF_LR_&_SVM\\Dataset_TFIDF_LR_&_SVM.csv", 
                   index=False)